In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity-ml-compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cluster = ComputeTarget(workspace=ws,name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_v2",max_nodes=4)
    cluster = ComputeTarget.create(ws,cluster_name,compute_config)
    cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(0.1, 4.0),
    '--max_iter': choice(50, 100, 150, 200, 250, 300)
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=6)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig('./', script='train.py', compute_target=cluster, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=8,
                                    max_concurrent_runs=4,
                                    run_config=src)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

In [ ]:
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])

print(best_run_parameter_values)

In [ ]:
# Just to make sure we understand HyperDrive AutoML Concept more, we will do another
# experiment where we maximize F1 Score and try to find if there is a better model or not?
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='F1_score',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=8,
                                    max_concurrent_runs=4,
                                    run_config=src)

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")

best_run

In [ ]:
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('F1_score:', best_run_metrics['F1_score'])

print(best_run_parameter_values)

In [ ]:
# So basically we have now 2 runs for AutoML Hyperdrive:
# 1. Run 1: Primary metric as Accuracy with score of 91%
# 2. Run 2: Attempted Primary Metric of F1-Score with score of 0.51

# Based on these 2 experiments, we conclude that our final AutoML Config should look for 'Accuracy' as our primary metric.

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
df = ds.to_pandas_dataframe(ds)
df.head()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=cluster,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_run_job, best_automl_model = automl_run.get_output()
joblib.dump(value=best_automl_model, filename='automl_model.joblib')

In [ ]:
best_automl_model

In [ ]:
automl_run_job

In [ ]:
cluster.delete()